In [2]:
%cd ~/protein-transfer

/home/francesca/protein-transfer


In [3]:
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

In [4]:
from scr.utils import pickle_load
from scr.analysis.utils import METRIC_DICT, metric_simplifier
from scr.analysis.perlayer import LayerLoss

In [42]:
sklearn_carp = LayerLoss(
    input_path="results/sklearn-carp",
    output_path="results/sklearn-carp_layer"
)

Plotting collage_name proeng_aav_one_vs_many_mean...
Plotting collage_name proeng_aav_two_vs_many_mean...
Plotting collage_name proeng_gb1_low_vs_high_mean...
Plotting collage_name proeng_gb1_two_vs_rest_mean...
Plotting collage_name proeng_gb1_sampled_mean...
Plotting collage_name proeng_thermo_mixed_split_mean...


In [ ]:
pytorch_carp = LayerLoss(
    input_path="results/pytorch-carp",
    output_path="results/pytorch-carp_layer"
)

Plotting collage_name annotation_scl_balanced_mean...
Plotting collage_name structure_ss3_tape_processed_noflatten...


In [5]:
import pandas as pd

In [7]:
pytorch_carp.layer_analysis_dict.keys(), sklearn_carp.layer_analysis_dict.keys()

(dict_keys(['annotation_scl_balanced_mean', 'structure_ss3_tape_processed_noflatten']),
 dict_keys(['proeng_aav_one_vs_many_mean', 'proeng_aav_two_vs_many_mean', 'proeng_gb1_low_vs_high_mean', 'proeng_gb1_two_vs_rest_mean', 'proeng_gb1_sampled_mean', 'proeng_thermo_mixed_split_mean']))

In [8]:
pytorch_carp.layer_analysis_dict[
    "annotation_scl_balanced_mean"
].keys(), sklearn_carp.layer_analysis_dict["proeng_aav_one_vs_many_mean"].keys()

(dict_keys(['carp_600k', 'carp_76M', 'carp_38M', 'carp_640M']),
 dict_keys(['carp_640M', 'carp_76M', 'carp_600k', 'carp_38M']))

In [9]:
pytorch_carp.layer_analysis_dict["annotation_scl_balanced_mean"][
    "carp_600k"
].keys(), sklearn_carp.layer_analysis_dict["proeng_aav_one_vs_many_mean"][
    "carp_600k"
].keys()

(dict_keys(['train_cross-entropy', 'val_cross-entropy', 'test_cross-entropy', 'test_acc', 'test_rocauc']),
 dict_keys(['train_mse', 'val_mse', 'test_mse', 'test_ndcg', 'test_rho']))

In [10]:
[
    metric_simplifier(m)
    for m in pytorch_carp.layer_analysis_dict["annotation_scl_balanced_mean"][
        "carp_600k"
    ].keys()
]

['train_loss',
 'val_loss',
 'test_loss',
 'test_performance_2',
 'test_performance_1']

In [11]:
[
    metric_simplifier(m)
    for m in sklearn_carp.layer_analysis_dict["proeng_aav_one_vs_many_mean"][
        "carp_600k"
    ].keys()
]

['train_loss',
 'val_loss',
 'test_loss',
 'test_performance_1',
 'test_performance_2']

In [30]:
master_results = pd.DataFrame(columns=["arch", "task", "model", "ablation", "metric", ])
master_results

,arch,task,model,ablation,metric


In [52]:
layerloss = sklearn_carp

ablation_list = ["emb", "rand", "stat", "onehot"]
ablation_dict_list = [
    layerloss.layer_analysis_dict,
    layerloss.rand_layer_analysis_dict,
    layerloss.stat_layer_analysis_dict,
    layerloss.onehot_baseline_dict,
]

In [54]:
arch = "carp"

# add checkpoints
if arch == "carp":
    cp_list = list(layerloss.checkpoint_analysis_dict.keys())
    ablation_list += [f"{arch}-{str(cp)}" for cp in cp_list]
    ablation_dict_list += [layerloss.checkpoint_analysis_dict[cp] for cp in cp_list]

for (ablation, ablation_dict) in zip(ablation_list, ablation_dict_list):
    for task in ablation_dict.keys():
        for model in ablation_dict[task].keys():
            for metric in ablation_dict[task][model].keys():
                master_results = pd.concat(
                    [
                        master_results,
                        pd.DataFrame(
                            {
                                "arch": arch,
                                "task": task,
                                "model": model,
                                "ablation": ablation,
                                "metric": metric_simplifier(metric),
                                "value": [
                                    ablation_dict[task][model][metric]
                                ],
                            }
                        ),
                    ],
                    ignore_index=True,
                )

In [55]:
master_results

,arch,task,model,ablation,metric,value
0,carp,proeng_aav_one_vs_many_mean,carp_640M,emb,train_loss,"[9.59526940116963, 6.894266263718891, 6.377686..."
1,carp,proeng_aav_one_vs_many_mean,carp_640M,emb,val_loss,"[11.18654191999164, 8.622644328345816, 8.33617..."
2,carp,proeng_aav_one_vs_many_mean,carp_640M,emb,test_loss,"[13.582533778622887, 53.712009437112215, 74.04..."
3,carp,proeng_aav_one_vs_many_mean,carp_640M,emb,test_performance_1,"[0.9704321391244177, 0.9629028051372428, 0.962..."
4,carp,proeng_aav_one_vs_many_mean,carp_640M,emb,test_performance_2,"[0.19544242309285326, 0.3282696048440418, 0.31..."
...,...,...,...,...,...,...
2335,carp,proeng_thermo_mixed_split_mean,carp_600k,carp-0.125,train_loss,"[2772.63146923264, 2770.5755982554224, 2769.87..."
2336,carp,proeng_thermo_mixed_split_mean,carp_600k,carp-0.125,val_loss,"[2784.692149993736, 2783.717326581634, 2784.13..."
2337,carp,proeng_thermo_mixed_split_mean,carp_600k,carp-0.125,test_loss,"[2830.026468153979, 2825.216248772877, 2823.48..."
2338,carp,proeng_thermo_mixed_split_mean,carp_600k,carp-0.125,test_performance_1,"[0.9820406873350307, 0.9852848539428993, 0.985..."


In [29]:
# pd.json_normalize(pytorch_carp.layer_analysis_dict)

In [8]:
pytorch_carp.layer_analysis_dict["annotation_scl_balanced_mean"]["carp_600k"]["train_cross-entropy"]

array([1.87699079, 1.75273726, 1.69514218, 1.60551146, 1.53652087,
       1.46996107, 1.40448275, 1.33026753, 1.30898465, 1.26282578,
       1.26654993, 1.21502749, 1.19079669, 1.183124  , 1.19900847,
       1.17011334, 1.18096242])

In [53]:
pd.DataFrame.from_dict(pytorch_carp.checkpoint_analysis_dict).T

,annotation_scl_balanced_mean,structure_ss3_tape_processed_noflatten
0.500,{'carp_600k': {'train_cross-entropy': [1.86432...,{'carp_76M': {'train_cross-entropy': [1.039851...
0.250,{'carp_600k': {'train_cross-entropy': [1.85454...,{'carp_76M': {'train_cross-entropy': [1.040206...
0.125,{'carp_600k': {'train_cross-entropy': [1.84984...,{'carp_76M': {'train_cross-entropy': [1.040783...


In [41]:
pd.DataFrame.from_dict(pytorch_carp.onehot_baseline_dict)

,annotation_scl_balanced,structure_ss3_tape_processed
train_cross-entropy,[0.3524752729817441],[1.0096537658613023]
val_cross-entropy,[1.373852210385459],[1.0060921719199734]
test_cross-entropy,[2.0159953236579895],NaN
test_acc,[0.37402597402597404],NaN
test_rocauc,[0.6619061060529179],NaN
casp12_acc,NaN,[0.48194597574421166]
casp12_rocauc,NaN,[0.6202125329637339]
cb513_acc,NaN,[0.48816846771958994]
cb513_rocauc,NaN,[0.6405190154371231]
ts115_acc,NaN,[0.5085510368973876]


In [47]:
pd.DataFrame.from_dict(sklearn_carp.onehot_baseline_dict)

,proeng_aav_one_vs_many,proeng_aav_two_vs_many,proeng_gb1_low_vs_high,proeng_gb1_two_vs_rest,proeng_gb1_sampled,proeng_thermo_mixed_split
train_mse,[6.606138986229867],[16.67168599756062],[0.1770679055989013],[2.090415559759884],[2.0079257240201596],[2760.1282354723244]
val_mse,[8.361448368900476],[16.951583715058266],[0.17003734650743707],[2.0447116896087842],[1.9726564028720246],[2778.184268114801]
test_mse,[66.71775739461496],[24.814891839771693],[4.028476326030474],[5.335454660039244],[2.025209681370326],[2838.811461721218]
test_ndcg,[0.960145115831988],[0.9629290404042454],[0.9178409557926928],[0.8905737288756033],[0.9342600571731149],[0.9554623184824335]
test_rho,[0.1903477426748507],[-0.0015626678198874598],[0.32173083298452543],[0.5428396414906184],[0.7885047525700145],[0.1226657473529901]


In [8]:
pd.DataFrame.from_dict(pytorch_carp.layer_analysis_dict)

,annotation_scl_balanced_mean,structure_ss3_tape_processed_noflatten
carp_600k,"{'train_cross-entropy': [1.8769907888613249, 1...","{'train_cross-entropy': [1.0358617109795139, 0..."
carp_76M,"{'train_cross-entropy': [1.71919468202089, 1.4...","{'train_cross-entropy': [1.0396245995612994, 0..."
carp_38M,"{'train_cross-entropy': [1.7385239789360447, 1...","{'train_cross-entropy': [1.0429124097301536, 0..."
carp_640M,"{'train_cross-entropy': [1.7193063999477185, 1...","{'train_cross-entropy': [1.0420560608171436, 0..."


In [20]:
pd.json_normalize(pd.DataFrame.from_dict(pytorch_carp.layer_analysis_dict)['annotation_scl_balanced_mean'])

,train_cross-entropy,val_cross-entropy,test_cross-entropy,test_acc,test_rocauc
0,"[1.8769907888613249, 1.7527372648841457, 1.695...","[1.9061075278690882, 1.7826685564858573, 1.727...","[1.9521732330322266, 1.9269767999649048, 1.910...","[0.2623376623376623, 0.2753246753246753, 0.316...","[0.6495972923926321, 0.6703754074198381, 0.673..."
1,"[1.71919468202089, 1.4959022904697217, 1.44056...","[1.7575252737317766, 1.534557512828282, 1.4600...","[2.0076744556427, 2.0575806498527527, 1.995046...","[0.3038961038961039, 0.35324675324675325, 0.35...","[0.6315851180553764, 0.7013079524965977, 0.710..."
2,"[1.7385239789360447, 1.4665500490288985, 1.395...","[1.7737009525299072, 1.5041547673089164, 1.417...","[2.035592496395111, 2.0621028542518616, 2.0096...","[0.2857142857142857, 0.36883116883116884, 0.37...","[0.6189812048918136, 0.7025105662414319, 0.713..."
3,"[1.7193063999477185, 1.5111642197558754, 1.437...","[1.7644925117492676, 1.5379446830068315, 1.462...","[2.0392618775367737, 2.05305552482605, 2.01585...","[0.2961038961038961, 0.34805194805194806, 0.36...","[0.6273352139525914, 0.7005424935091661, 0.706..."


In [ ]:
pd.json_normalize(pd.DataFrame.from_dict(pytorch_carp.layer_analysis_dict)['annotation_scl_balanced_mean'])

In [21]:
pd.json_normalize(pd.DataFrame.from_dict(sklearn_carp.layer_analysis_dict)['proeng_aav_one_vs_many_mean'])

,train_mse,val_mse,test_mse,test_ndcg,test_rho
0,"[9.59526940116963, 6.894266263718891, 6.377686...","[11.18654191999164, 8.622644328345816, 8.33617...","[13.582533778622887, 53.712009437112215, 74.04...","[0.9704321391244177, 0.9629028051372428, 0.962...","[0.19544242309285326, 0.3282696048440418, 0.31..."
1,"[9.582026132905252, 6.934608975116892, 6.40550...","[11.20643614220332, 8.899376014238195, 8.20574...","[13.697288684643041, 48.961780681241656, 69.96...","[0.9703413636783854, 0.9633589617754629, 0.962...","[0.2116112794657769, 0.3396909252934983, 0.320..."
2,"[9.468360495727067, 8.190111410830674, 7.61644...","[10.9554842436175, 9.782055506244191, 9.209345...","[14.20679802985108, 24.53599377907496, 23.3161...","[0.9698271299324172, 0.96692411292824, 0.96943...","[0.23682032708767906, 0.3808350241317865, 0.43..."
3,"[9.574777226192223, 6.690699098364959, 6.35389...","[11.175331793579538, 8.655740728556914, 8.5822...","[13.968634660845268, 65.68949789195031, 78.864...","[0.969236054842432, 0.9626272649406287, 0.9625...","[0.18915018144628126, 0.3242017861013232, 0.31..."


In [18]:
pd.DataFrame.from_dict(sklearn_carp.layer_analysis_dict)

,proeng_aav_one_vs_many_mean,proeng_aav_two_vs_many_mean,proeng_gb1_low_vs_high_mean,proeng_gb1_two_vs_rest_mean,proeng_gb1_sampled_mean,proeng_thermo_mixed_split_mean
carp_640M,"{'train_mse': [9.59526940116963, 6.89426626371...","{'train_mse': [20.068343175878088, 16.92011420...","{'train_mse': [0.2772588428285639, 0.408520416...","{'train_mse': [2.725838148323684, 1.9707604225...","{'train_mse': [2.5043217133381526, 1.876857337...","{'train_mse': [2772.749931574079, 2768.7387720..."
carp_76M,"{'train_mse': [9.582026132905252, 6.9346089751...","{'train_mse': [19.986434661076075, 16.91308441...","{'train_mse': [0.2853286156211369, 0.411029697...","{'train_mse': [2.7027404465889115, 1.969533968...","{'train_mse': [2.4793497727220144, 1.871709821...","{'train_mse': [2772.772707139839, 2768.7354513..."
carp_600k,"{'train_mse': [9.468360495727067, 8.1901114108...","{'train_mse': [19.808120463435134, 17.93898064...","{'train_mse': [0.280335127217654, 0.3869098418...","{'train_mse': [2.674702299445244, 2.1214922598...","{'train_mse': [2.4329909760979573, 2.039620513...","{'train_mse': [2772.624627474502, 2770.3332577..."
carp_38M,"{'train_mse': [9.574777226192223, 6.6906990983...","{'train_mse': [20.04262815315973, 16.802693836...","{'train_mse': [0.27452013074623355, 0.40600096...","{'train_mse': [2.7247057205531227, 1.959347953...","{'train_mse': [2.507311420931766, 1.8454394133...","{'train_mse': [2772.7840732976197, 2768.293532..."


In [28]:
pd.merge(
    pd.DataFrame.from_dict(sklearn_carp.layer_analysis_dict),
    pd.DataFrame.from_dict(pytorch_carp.layer_analysis_dict),
    left_index=True,
    right_index=True,
).to_csv("results/summary/all_carp.csv")

In [29]:
pd.merge(
    pd.DataFrame.from_dict(sklearn_carp.rand_layer_analysis_dict),
    pd.DataFrame.from_dict(pytorch_carp.rand_layer_analysis_dict),
    left_index=True,
    right_index=True,
).to_csv("results/summary/all_carp_rand.csv")

In [30]:
pd.merge(
    pd.DataFrame.from_dict(sklearn_carp.stat_layer_analysis_dict),
    pd.DataFrame.from_dict(pytorch_carp.stat_layer_analysis_dict),
    left_index=True,
    right_index=True,
).to_csv("results/summary/all_carp_stat.csv")

In [42]:
pd.merge(
    pd.DataFrame.from_dict(sklearn_carp.onehot_baseline_dict),
    pd.DataFrame.from_dict(pytorch_carp.onehot_baseline_dict),
    left_index=True,
    right_index=True,
).to_csv("results/summary/all_carp_oh.csv")

In [50]:
pd.DataFrame.from_dict(sklearn_carp.onehot_baseline_dict).merge(
    pd.DataFrame.from_dict(pytorch_carp.onehot_baseline_dict),
    how="outer",
    left_index=True,
    right_index=True,
).fillna(False).to_csv("results/summary/all_carp_oh.csv")

In [54]:
pd.merge(
    pd.DataFrame.from_dict(sklearn_carp.checkpoint_analysis_dict).T,
    pd.DataFrame.from_dict(pytorch_carp.checkpoint_analysis_dict).T,
    left_index=True,
    right_index=True,
).to_csv("results/summary/all_carp_cp.csv")

In [34]:
pytorch_esm = LayerLoss(
    input_path="results/pytorch-esm",
    output_path="results/pytorch-esm_layer",
    add_checkpoint=False
)

Plotting collage_name structure_ss3_tape_processed_noflatten...
Plotting collage_name annotation_scl_balanced_mean...


In [35]:
sklearn_esm = LayerLoss(
    input_path="results/sklearn-esm",
    output_path="results/sklearn-esm_layer",
    add_checkpoint=False
)

Plotting collage_name proeng_thermo_mixed_split_mean...
Plotting collage_name proeng_gb1_two_vs_rest_mean...
Plotting collage_name proeng_gb1_low_vs_high_mean...
Plotting collage_name proeng_gb1_sampled_mean...
Plotting collage_name proeng_aav_one_vs_many_mean...
Plotting collage_name proeng_aav_two_vs_many_mean...


In [36]:
pd.merge(
    pd.DataFrame.from_dict(sklearn_esm.layer_analysis_dict),
    pd.DataFrame.from_dict(pytorch_esm.layer_analysis_dict),
    left_index=True,
    right_index=True,
).to_csv("results/summary/all_esm.csv")

In [37]:
pd.merge(
    pd.DataFrame.from_dict(sklearn_esm.rand_layer_analysis_dict),
    pd.DataFrame.from_dict(pytorch_esm.rand_layer_analysis_dict),
    left_index=True,
    right_index=True,
).to_csv("results/summary/all_esm_rand.csv")

In [38]:
pd.merge(
    pd.DataFrame.from_dict(sklearn_esm.stat_layer_analysis_dict),
    pd.DataFrame.from_dict(pytorch_esm.stat_layer_analysis_dict),
    left_index=True,
    right_index=True,
).to_csv("results/summary/all_esm_stat.csv")

In [51]:
pd.DataFrame.from_dict(sklearn_esm.onehot_baseline_dict).merge(
    pd.DataFrame.from_dict(pytorch_esm.onehot_baseline_dict),
    how="outer",
    left_index=True,
    right_index=True,
).fillna(False).to_csv("results/summary/all_esm_oh.csv")

In [ ]:
class LayerLoss2CSV(LayerLoss):
    """A class for reorganizing layer by layer results into one csv"""

    def __init__(
        self,
        add_checkpoint: bool = True,
        checkpoint_list: list = [0.5, 0.25, 0.125],
        input_path: str = "results/sklearn",
        output_path: str = "results/sklearn_layer",
        metric_dict: dict[list[str]] = {
            "proeng": ["train_mse", "val_mse", "test_mse", "test_ndcg", "test_rho"],
            "annotation": [
                "train_cross-entropy",
                "val_cross-entropy",
                "test_cross-entropy",
                "test_acc",
                "test_rocauc",
            ],
            "structure": [
                "train_cross-entropy",
                "val_cross-entropy",
                "casp12_acc",
                "casp12_rocauc",
                "cb513_acc",
                "cb513_rocauc",
                "ts115_acc",
                "ts115_rocauc",
            ],
        },
    ):
        """
        Args:
        - add_checkpoint: bool = True, if add checkpoint for carp
        - checkpoint_list: list = [0.5, 0.25, 0.125],
        - input_path: str = "results/sklearn",
        - output_path: str = "results/sklearn_layer"
        - metric_dict: list[str] = ["train_mse", "test_ndcg", "test_rho"]
        """


In [11]:
all_results_df = pd.read_csv("results/summary/all_results.csv")

In [12]:
all_results_df["value"]

0       [9.59526940116963, 6.894266263718891, 6.377686...
1       [11.18654191999164, 8.622644328345816, 8.33617...
2       [13.582533778622887, 53.712009437112215, 74.04...
3       [0.9704321391244177, 0.9629028051372428, 0.962...
4       [0.19544242309285326, 0.3282696048440418, 0.31...
                              ...                        
1629                                 [0.3524752729817441]
1630                                  [1.373852210385459]
1631                                 [2.0159953236579895]
1632                                [0.37402597402597404]
1633                                 [0.6619061060529179]
Name: value, Length: 1634, dtype: object